In [8]:
import json
import re
import os

def fix_json(json_string):
    json_string = json_string.strip()
    if not json_string.startswith('['):
        json_string = '[' + json_string
    if not json_string.endswith(']'):
        json_string = json_string + ']'
    json_string = re.sub(r'("(?:[^"\\]|\\.)*")', lambda m: m.group(1).replace('\n', '\\n'), json_string)
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)
    return json_string

def extract_final_answer(gpt_result):
    match = re.search(r'Final Answer:\s*([A-J])', gpt_result)
    return match.group(1) if match else None

def process_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    fixed_content = fix_json(content)
    
    try:
        data = json.loads(fixed_content)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None, 0
    
    correct_count = 0
    total_count = 0
    
    for item in data:
        gpt_result = item.get('gpt_result', '')
        correct_answer = item.get('correct_answer', '')
        
        extracted_answer = extract_final_answer(gpt_result)
        
        if extracted_answer:
            item['final_answer'] = ord(extracted_answer) - ord('A')
            item['is_correct'] = (extracted_answer == correct_answer)
            if item['is_correct']:
                correct_count += 1
        else:
            item['final_answer'] = -1
            item['is_correct'] = False
        
        total_count += 1
    
    accuracy = correct_count / total_count if total_count > 0 else 0
    
    return data, accuracy

# Process the file
file_path = '/Users/jongg21/Desktop/LEAN/Z3-Solver/mmlupro.json'
processed_data, accuracy = process_json_file(file_path)

if processed_data:
    # Print the results
    print(f"Accuracy: {accuracy:.2%}")

    # Save the processed data to a new JSON file with "_updated" appended
    file_name, file_extension = os.path.splitext(file_path)
    new_file_path = f"{file_name}_updated{file_extension}"
    with open(new_file_path, 'w', encoding='utf-8') as file:
        json.dump(processed_data, file, indent=2)
    print(f"Updated data saved to: {new_file_path}")
else:
    print("Failed to process the JSON file.")

Accuracy: 76.61%
Updated data saved to: /Users/jongg21/Desktop/LEAN/Z3-Solver/mmlupro_updated.json
